In [5]:
import torch
import clip
from PIL import Image
import os
import numpy as np

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

In [ ]:
embeddings = []
filenames = []
img_path = "data"
for img_file in os.listdir(img_path):
    if img_file.lower().endswith((".jpg", ".jpeg", ".png")):
        img = preprocess(Image.open(os.path.join(img_path, img_file))).unsqueeze(0).to(device)
        with torch.no_grad():
            embedding = model.encode_image(img)
            embedding = embedding / embedding.norm(dim=-1, keepdim=True)
            embeddings.append(embedding.cpu().numpy().flatten())
            filenames.append(img_file)

In [8]:
embeddings = np.stack(embeddings)
correlation_matrix = embeddings @ embeddings.T

print("Correlation Matrix:")
print(correlation_matrix)
print("Best matches (excluding self-matches):")
for i in range(len(filenames)):
    best_match = np.argsort(correlation_matrix[i])[-2]
    print(f"{filenames[i]} <-> {filenames[best_match]}: {correlation_matrix[i, best_match]:.4f}")

Correlation Matrix:
[[0.9999999  0.88214666 0.73616123 0.73968023 0.6835071  0.71770716
  0.78977036 0.777936   0.7922236 ]
 [0.88214666 0.9999999  0.7211038  0.722103   0.6934829  0.7454036
  0.81593037 0.78559554 0.79571706]
 [0.73616123 0.7211038  0.99999976 0.709729   0.68558013 0.71068364
  0.75588906 0.69562024 0.76012266]
 [0.73968023 0.722103   0.709729   1.0000002  0.7199095  0.7616646
  0.78797066 0.7141116  0.7549044 ]
 [0.6835071  0.6934829  0.68558013 0.7199095  1.0000002  0.78927624
  0.72272754 0.61904836 0.721447  ]
 [0.71770716 0.7454036  0.71068364 0.7616646  0.78927624 0.99999976
  0.8076376  0.6860337  0.7556789 ]
 [0.78977036 0.81593037 0.75588906 0.78797066 0.72272754 0.8076376
  0.9999999  0.7720922  0.84367037]
 [0.777936   0.78559554 0.69562024 0.7141116  0.61904836 0.6860337
  0.7720922  0.99999976 0.7686957 ]
 [0.7922236  0.79571706 0.76012266 0.7549044  0.721447   0.7556789
  0.84367037 0.7686957  0.9999999 ]]
Best matches (excluding self-matches):
0.jpg <->